In [1]:
import numpy as np

class ExperimentControl:
    def __init__(self):
        self.command_count = 0
        self.scan_count = 0
        self.total_shots = 0
        self.lines = []

    def append_command(self, device, variable, value, precision=4):
        """
        Appends a command line to the lines list.
        
        Parameters:
        - device (str): The device name.
        - variable (str): The variable name.
        - value (float): The value for the variable.
        - precision (int): The number of decimal places for the value.
        
        Returns:
        - int: The number of shots for this command (always 1).
        """
        self.command_count += 1
        value_str = f'{value:.{precision}f}'
        command_line = f'[Command{self.command_count}]\nDevice = "{device}"\nVariable = "{variable}"\nValue= "{value_str}"\n\n'
        self.lines.append(command_line)
        return 1  # Assuming a single shot for a command

    def append_nloop_scan(self, device, variable, start, end, step, shots_per_step, precision=12):
        """
        Appends an Nloop scan line to the lines list.
        
        Parameters:
        - device (str): The device name.
        - variable (str): The variable name.
        - start (float): The start value for the scan.
        - end (float): The end value for the scan.
        - step (float): The step size for the scan.
        - shots_per_step (int): The number of shots per step.
        - precision (int): The number of decimal places for the values.
        
        Returns:
        - int: The total number of shots for this Nloop scan.
        """
        self.scan_count += 1
        values = []
        num_shots = 0
        if start < end:
            current = start
            while current <= end:
                values.append(f'({current:.{precision}f}):{shots_per_step}')
                current += step
                num_shots += shots_per_step
        else:
            current = start
            while current >= end:
                values.append(f'({current:.{precision}f}):{shots_per_step}')
                current += step
                num_shots += shots_per_step
        
        values_str = '|'.join(values)
        scan_line = f'[Scan{self.scan_count}]\nDevice = "{device}"\nVariable = "{variable}"\nValues:#shots = "{values_str}"\n\n'
        self.lines.append(scan_line)
        self.total_shots += num_shots
        return num_shots

    def append_no_scan(self, device, variable, value, shots, precision=12):
        """
        Appends a no-scan line to the lines list.
        
        Parameters:
        - device (str): The device name.
        - variable (str): The variable name.
        - value (float): The value for the variable.
        - shots (int): The number of shots.
        - precision (int): The number of decimal places for the value.
        
        Returns:
        - int: The number of shots for this no scan.
        """
        self.scan_count += 1
        value_str = f'({value:.{precision}f}):{shots}|'
        scan_line = f'[Scan{self.scan_count}]\nDevice = "{device}"\nVariable = "{variable}"\nValues:#shots = "{value_str}"\n\n'
        self.lines.append(scan_line)
        self.total_shots += shots
        return shots

    def append_nmloop_scan(self, device1, device2, variable1, variable2, values, shots_per_value, precision=12):
        """
        Appends an NMloop scan line to the lines list.
        
        Parameters:
        - device1 (str): The first device name.
        - device2 (str): The second device name.
        - variable1 (str): The first variable name.
        - variable2 (str): The second variable name.
        - values (list of tuples): The values for the NMloop scan.
        - shots_per_value (int): The number of shots per value pair.
        - precision (int): The number of decimal places for the values.
        
        Returns:
        - int: The total number of shots for this NMloop scan.
        """
        self.scan_count += 1
        values_str = '|'.join([f'({v1:.{precision}f},{v2:.{precision}f}):{shots_per_value}' for v1, v2 in values])
        scan_line = f'[Scan{self.scan_count}]\nDevice = "{device1},{device2}"\nVariable = "{variable1},{variable2}"\nValues:#shots = "{values_str}"\n\n'
        self.lines.append(scan_line)
        num_shots = shots_per_value * len(values)
        self.total_shots += num_shots
        return num_shots

    def append_nmloop_scan_with_poly(self, device1, device2, variable1, variable2, start, end, step, p, shots_per_value, precision=12):
        """
        Appends an NMloop scan line to the lines list using a polynomial relationship.
        
        Parameters:
        - device1 (str): The first device name.
        - device2 (str): The second device name.
        - variable1 (str): The first variable name.
        - variable2 (str): The second variable name.
        - start (float): The start value for the first device.
        - end (float): The end value for the first device.
        - step (float): The step size for the first device.
        - p (list of float): The polynomial coefficients for the relationship.
        - shots_per_value (int): The number of shots per value pair.
        - precision (int): The number of decimal places for the values.
        
        Returns:
        - int: The total number of shots for this NMloop scan.
        """
        self.scan_count += 1
        values = []
        current = start
        num_shots = 0

        while (start < end and current <= end) or (start > end and current >= end):
            device2_value = np.polyval(p, current)
            values.append((current, device2_value))
            current += step
            num_shots += shots_per_value

        values_str = '|'.join([f'({v1:.{precision}f},{v2:.{precision}f}):{shots_per_value}' for v1, v2 in values])
        scan_line = f'[Scan{self.scan_count}]\nDevice = "{device1},{device2}"\nVariable = "{variable1},{variable2}"\nValues:#shots = "{values_str}"\n\n'
        self.lines.append(scan_line)
        self.total_shots += num_shots
        return num_shots

    def write_to_file(self, filename):
        """
        Writes the lines to a text file.
        
        Parameters:
        - filename (str): The name of the file to write to.
        """
        with open(filename, 'w') as file:
            file.writelines(self.lines)
            
    def run_experiment(self, pressure_value, nloop_device, nloop_variable, nloop_start, nloop_end, nloop_step, nloop_shots_per_step, precision=12):
        """
        Runs the experiment sequence, appending all necessary commands and scans.
        
        Parameters:
        - pressure_value (float): The pressure value to set.
        - nloop_device (str): The device name for the Nloop scan.
        - nloop_variable (str): The variable name for the Nloop scan.
        - nloop_start (float): The start value for the Nloop scan.
        - nloop_end (float): The end value for the Nloop scan.
        - nloop_step (float): The step size for the Nloop scan.
        - nloop_shots_per_step (int): The number of shots per step for the Nloop scan.
        - precision (int): The number of decimal places for the values.
        
        Returns:
        - int: The total number of shots for this experiment sequence.
        """
        total_shots = 0
        # Background scan
        total_shots += self.append_no_scan("HTT_LowPressureControl", "pressure", 0, 20, precision)

        # Set pressure to specified value
        total_shots += self.append_command("HTT_LowPressureControl", "pressure", pressure_value, precision)

        # Nloop scan
        total_shots += self.append_nloop_scan(nloop_device, nloop_variable, nloop_start, nloop_end, nloop_step, nloop_shots_per_step, precision)

        # Set pressure back to 0
        total_shots += self.append_command("HTT_LowPressureControl", "pressure", 0, precision)

        # Background scan
        total_shots += self.append_no_scan("HTT_LowPressureControl", "pressure", 0, 20, precision)
        
        return total_shots

    def calculate_total_time(self, experiment_rep_rate):
        """
        Calculates and prints the total time taken for the experiment.
        
        Parameters:
        - experiment_rep_rate (float): The experiment repetition rate in Hz.
        """
        total_time = self.total_shots / experiment_rep_rate
        print(f"Total time taken for the experiment: {total_time} seconds")


In [2]:
# Example usage:
experiment_control = ExperimentControl()

# Define your experiments
experiments = [
    {"pressure_value": 80, 
     "nloop_device": "HTT_LowPressureControl", 
     "nloop_variable": "pressure", 
     "nloop_start": 0.0, 
     "nloop_end": 100.0, 
     "nloop_step": 10, 
     "nloop_shots_per_step": 10
    },
    # {"pressure_value": 90, "nloop_device": "DeviceB", "nloop_variable": "VariableB", "nloop_start": 0.2, "nloop_end": 2.0, "nloop_step": 0.2, "nloop_shots_per_step": 10}
]

# Run the experiments
for experiment in experiments:
    experiment_control.run_experiment(**experiment, precision=10)

# Calculate total time taken for the experiment
experiment_control.calculate_total_time(experiment_rep_rate=1)  # Example repetition rate of 10 Hz

# Writing to a text file
experiment_control.write_to_file(r'Z:\data\Thomson\custom_scans\test python experiment_control.txt')


Total time taken for the experiment: 150.0 seconds


In [7]:
# Example usage:
experiment_control = ExperimentControl()

gas_cell_pinhole_vals = [6.5, 7.0, 7.5]

# Define your experiments
experiments = [
    {"pressure_value": 80, 
     "nloop_device": "HTT-Aerotech", 
     "nloop_variable": "drive grating 2 trans", 
     "nloop_start": 49830, 
     "nloop_end": 52430, 
     "nloop_step": 200, 
     "nloop_shots_per_step": 10
    },
    # {"pressure_value": 90, "nloop_device": "DeviceB", "nloop_variable": "VariableB", "nloop_start": 0.2, "nloop_end": 2.0, "nloop_step": 0.2, "nloop_shots_per_step": 10}
]

# Run the experiments
for i in range(len(gas_cell_pinhole_vals)):
    experiment = experiments[0]
    experiment_control.append_command("HTT_GasCellFrontPinhole", "position", gas_cell_pinhole_vals[i], precision=4)
    experiment_control.run_experiment(**experiment, precision=10)
    
# Calculate total time taken for the experiment
experiment_control.calculate_total_time(experiment_rep_rate=1)  # Example repetition rate of 10 Hz

# Writing to a text file
experiment_control.write_to_file(r'Z:\data\Thomson\custom_scans\2024-06-05 compression pinhole 2D scan w bgs.txt')


Total time taken for the experiment: 540.0 seconds


In [10]:
experiment_control = ExperimentControl()
experiment_control.append_no_scan("HTT-Aerotech", "drive grating 2 trans", 52430, shots, precision=12)


NameError: name 'device' is not defined